In [32]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

## 전처리된 수도권 데이터에서 출발행정동과 도착 행정동의 위도,경도 추출 

In [34]:
# 수도권 OD CSV 파일 불러오기
df = pd.read_csv("metropoli_final.csv", encoding="utf-8-sig")
df.head()

,출발 행정동 코드,도착 행정동 코드,출발 행정동 주소,도착 행정동 주소
0,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동
1,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동
2,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동
3,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동
4,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동


In [92]:
df.shape

(14168294, 4)

In [5]:
# 조건에 맞는 시/도 목록
valid_regions = ("서울특별시", "경기도", "인천광역시")

In [7]:
# 조건: 출발지와 도착지 주소 모두 valid_regions로 시작하는 경우
df = df[
    df["출발 행정동 주소"].str.startswith(valid_regions) &
    df["도착 행정동 주소"].str.startswith(valid_regions)
]

# 결과 확인
print(df.head())


   Unnamed: 0  출발 행정동 코드  도착 행정동 코드      출발 행정동 주소      도착 행정동 주소
0           0    1101053    1101053  서울특별시 종로구 사직동  서울특별시 종로구 사직동
1           1    1101053    1101053  서울특별시 종로구 사직동  서울특별시 종로구 사직동
2           2    1101053    1101053  서울특별시 종로구 사직동  서울특별시 종로구 사직동
3           3    1101053    1101053  서울특별시 종로구 사직동  서울특별시 종로구 사직동
4           4    1101053    1101053  서울특별시 종로구 사직동  서울특별시 종로구 사직동


In [11]:
df = df.drop(columns=["Unnamed: 0"])
print(df.head())
print(df.shape)

   출발 행정동 코드  도착 행정동 코드      출발 행정동 주소      도착 행정동 주소
0    1101053    1101053  서울특별시 종로구 사직동  서울특별시 종로구 사직동
1    1101053    1101053  서울특별시 종로구 사직동  서울특별시 종로구 사직동
2    1101053    1101053  서울특별시 종로구 사직동  서울특별시 종로구 사직동
3    1101053    1101053  서울특별시 종로구 사직동  서울특별시 종로구 사직동
4    1101053    1101053  서울특별시 종로구 사직동  서울특별시 종로구 사직동
(14168294, 4)


In [64]:
# Kakao REST API 키 받기
api_key = "4ffdaedd97065f6660c65fe7c75d42e6"  # 예: "69e904e85329cddd6795bf2cfe6c576a"

In [66]:
# 3. 주소 → 위도/경도 요청 함수
def get_lat_lng(address, api_key):
    try:
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        headers = {"Authorization": f"KakaoAK {api_key}"}
        params = {"query": address}
        response = requests.get(url, headers=headers, params=params)

        print(f"[Query] {address}")
        print(f"[Status] {response.status_code}")
        print(f"[JSON] {response.json()}")  # 응답 확인

        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                location = result['documents'][0]['address']
                return location['y'], location['x']
        return None, None
    except Exception as e:
        print(f"[Exception] {e}")
        return None, None

In [40]:
# 출발 주소, 도착 주소 추출 
start_addresses = df["출발 행정동 주소"].tolist()  # 인덱스 
end_addresses = df['도착 행정동 주소'].tolist()
print(start_addresses[:5])
print(end_addresses[:5])

['서울특별시 종로구 사직동', '서울특별시 종로구 사직동', '서울특별시 종로구 사직동', '서울특별시 종로구 사직동', '서울특별시 종로구 사직동']
['서울특별시 종로구 사직동', '서울특별시 종로구 사직동', '서울특별시 종로구 사직동', '서울특별시 종로구 사직동', '서울특별시 종로구 사직동']


In [42]:
# 출발, 도착 주소 수도권에 있는지 재확인 
regions = ['경기도', '서울특별시', '인천광역시']

found = all(region in addr for addr in start_addresses for region in regions)

print("지역 포함 여부:", found)

지역 포함 여부: False


In [44]:
# unique한 주소 개수
print(len(set(start_addresses)))

1138


In [46]:
# 출발 주소 길이 
len(start_addresses)

14168294

In [68]:
#고유 주소만 추출 
unique_start_addresses = list(set(start_addresses))
unique_end_addresses = list(set(end_addresses))

In [69]:
# 캐시용 딕셔너리
s_address_to_coord = {}
e_address_to_coord = {}

In [86]:
# 출발 주소 병렬 처리
s_lat_list, s_lng_list = [], []

# 병렬 API 호출 (1138개만)
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {
        executor.submit(get_lat_lng, addr, api_key): addr
        for addr in unique_start_addresses
    }

    for future in tqdm(as_completed(futures), total=len(futures), desc="Geocoding unique addresses"):
        addr = futures[future]
        lat, lng = future.result()
        s_address_to_coord[addr] = (lat, lng)

# 출발 주소 리스트에 매핑
s_lat_list = []
s_lng_list = []

for addr in tqdm(start_addresses, desc="Applying cached coordinates"):
    lat, lng = s_address_to_coord.get(addr, (None, None))
    s_lat_list.append(lat)
    s_lng_list.append(lng)

Geocoding unique addresses:   0%|              | 1/1138 [00:00<03:38,  5.21it/s]

[Query] 서울특별시 강남구 일원본동[Query] 경기도 포천시 관인면
[Status] 200
[JSON] {'documents': [{'address': {'address_name': '경기 포천시 관인면', 'b_code': '4165040000', 'h_code': '4165040000', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '경기', 'region_2depth_name': '포천시', 'region_3depth_h_name': '관인면', 'region_3depth_name': '관인면', 'sub_address_no': '', 'x': '127.251198061906', 'y': '38.1577991234707'}, 'address_name': '경기 포천시 관인면', 'address_type': 'REGION', 'road_address': None, 'x': '127.251198061906', 'y': '38.1577991234707'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}
[Query] 인천광역시 부평구 부개3동
[Status] 200
[JSON] {'documents': [{'address': {'address_name': '인천 부평구 부개3동', 'b_code': '', 'h_code': '2823766100', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '인천', 'region_2depth_name': '부평구', 'region_3depth_h_name': '부개3동', 'region_3depth_name': '', 'sub_address_no': '', 'x': '126.734721955994', 'y': '37.5043177745185'}, 'address_name': '인천 부평구 부개3동', 'a

Geocoding unique addresses:   1%|▏            | 11/1138 [00:00<00:44, 25.11it/s]

[Query] 서울특별시 성동구 옥수동[Query] 경기도 평택시 원평동
[Status] 200
[JSON] {'documents': [{'address': {'address_name': '경기 평택시 원평동', 'b_code': '', 'h_code': '4122060000', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '경기', 'region_2depth_name': '평택시', 'region_3depth_h_name': '원평동', 'region_3depth_name': '', 'sub_address_no': '', 'x': '127.080400630556', 'y': '36.9892773148459'}, 'address_name': '경기 평택시 원평동', 'address_type': 'REGION', 'road_address': None, 'x': '127.080400630556', 'y': '36.9892773148459'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}
[Query] 경기도 수원시 장안구 정자2동
[Status] 200
[JSON] {'documents': [{'address': {'address_name': '경기 수원시 장안구 정자2동', 'b_code': '', 'h_code': '4111157200', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '경기', 'region_2depth_name': '수원시 장안구', 'region_3depth_h_name': '정자2동', 'region_3depth_name': '', 'sub_address_no': '', 'x': '127.000442105734', 'y': '37.295038054778'}, 'address_name': '경기 수원시 장안구 정자2동', 'ad

Geocoding unique addresses:   2%|▏            | 21/1138 [00:00<00:35, 31.54it/s]

...

Geocoding unique addresses:  98%|█████████▊| 1120/1138 [00:08<00:00, 132.28it/s]

[Query] 경기도 양평군 용문면[Query] 경기도 남양주시 퇴계원면
[Status] 200
[JSON] {'documents': [{'address': {'address_name': '경기 남양주시 퇴계원읍', 'b_code': '4136026500', 'h_code': '4136026500', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '경기', 'region_2depth_name': '남양주시', 'region_3depth_h_name': '퇴계원읍', 'region_3depth_name': '퇴계원읍', 'sub_address_no': '', 'x': '127.14191739863', 'y': '37.6474171947576'}, 'address_name': '경기 남양주시 퇴계원읍', 'address_type': 'REGION', 'road_address': None, 'x': '127.14191739863', 'y': '37.6474171947576'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

[Status] 200
[JSON] {'documents': [{'address': {'address_name': '경기 양평군 용문면', 'b_code': '4183040000', 'h_code': '4183040000', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '경기', 'region_2depth_name': '양평군', 'region_3depth_h_name': '용문면', 'region_3depth_name': '용문면', 'sub_address_no': '', 'x': '127.599161344744', 'y': '37.4889036515231'}, 'address_name': '경기 양평군 용문면', 'address_t

Geocoding unique addresses: 100%|██████████| 1138/1138 [00:09<00:00, 124.86it/s]


[Query] 경기도 광주시 남종면
[Status] 200
[JSON] {'documents': [{'address': {'address_name': '경기 광주시 남종면', 'b_code': '4161035000', 'h_code': '4161035000', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '경기', 'region_2depth_name': '광주시', 'region_3depth_h_name': '남종면', 'region_3depth_name': '남종면', 'sub_address_no': '', 'x': '127.301699267454', 'y': '37.494164363366'}, 'address_name': '경기 광주시 남종면', 'address_type': 'REGION', 'road_address': None, 'x': '127.301699267454', 'y': '37.494164363366'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}
[Query] 경기도 안양시 동안구 관양1동
[Status] 200
[JSON] {'documents': [{'address': {'address_name': '경기 안양시 동안구 관양동', 'b_code': '4117310200', 'h_code': '4117355200', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '경기', 'region_2depth_name': '안양시 동안구', 'region_3depth_h_name': '관양동', 'region_3depth_name': '관양동', 'sub_address_no': '', 'x': '126.961457538379', 'y': '37.40604574993'}, 'address_name': '경기 안양시 동안구 관양동', 'add

Applying cached coordinates: 100%|█| 14168294/14168294 [00:02<00:00, 6728774.27i


In [88]:
print(s_lat_list[:5])
print(s_lng_list[:5])

['37.5754222287938', '37.5754222287938', '37.5754222287938', '37.5754222287938', '37.5754222287938']
['126.96631967084', '126.96631967084', '126.96631967084', '126.96631967084', '126.96631967084']


In [78]:
# 도착 주소 병렬 처리
e_lat_list, e_lng_list = [], []

# 병렬 API 호출 (1138개만)
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {
        executor.submit(get_lat_lng, addr, api_key): addr
        for addr in unique_end_addresses
    }

    for future in tqdm(as_completed(futures), total=len(futures), desc="Geocoding unique addresses"):
        addr = futures[future]
        lat, lng = future.result()
        e_address_to_coord[addr] = (lat, lng)

# 도착 주소 리스트에 매핑
s_lat_list = []
s_lng_list = []

for addr in tqdm(end_addresses, desc="Applying cached coordinates"):
    lat, lng = e_address_to_coord.get(addr, (None, None))
    e_lat_list.append(lat)
    e_lng_list.append(lng)

Geocoding unique addresses:   2%|▎           | 28/1138 [00:00<00:08, 131.86it/s]

[Query] 서울특별시 영등포구 여의동
[Status] 200
[JSON] {'documents': [{'address': {'address_name': '서울 영등포구 여의동', 'b_code': '', 'h_code': '1156054000', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '영등포구', 'region_3depth_h_name': '여의동', 'region_3depth_name': '', 'sub_address_no': '', 'x': '126.934609800644', 'y': '37.5176044401505'}, 'address_name': '서울 영등포구 여의동', 'address_type': 'REGION', 'road_address': None, 'x': '126.934609800644', 'y': '37.5176044401505'}, {'address': {'address_name': '서울 영등포구 여의도동', 'b_code': '1156011000', 'h_code': '', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '영등포구', 'region_3depth_h_name': '', 'region_3depth_name': '여의도동', 'sub_address_no': '', 'x': '126.930167091136', 'y': '37.5267849150832'}, 'address_name': '서울 영등포구 여의도동', 'address_type': 'REGION', 'road_address': None, 'x': '126.930167091136', 'y': '37.5267849150832'}], 'meta': {'is_end': True, 'pageable_count': 2, 'total

Geocoding unique addresses:   5%|▋           | 61/1138 [00:00<00:07, 144.39it/s]

[Query] 서울특별시 광진구 광장동[Query] 경기도 부천시 원종1동
[Status] 200
[JSON] {'documents': [{'address': {'address_name': '경기 부천시 오정구 원종1동', 'b_code': '', 'h_code': '4119652000', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '경기', 'region_2depth_name': '부천시 오정구', 'region_3depth_h_name': '원종1동', 'region_3depth_name': '', 'sub_address_no': '', 'x': '126.805463777462', 'y': '37.5259749992067'}, 'address_name': '경기 부천시 오정구 원종1동', 'address_type': 'REGION', 'road_address': None, 'x': '126.805463777462', 'y': '37.5259749992067'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

[Status] 200
[JSON] {'documents': [{'address': {'address_name': '서울 광진구 광장동', 'b_code': '1121510400', 'h_code': '1121581000', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '광진구', 'region_3depth_h_name': '광장동', 'region_3depth_name': '광장동', 'sub_address_no': '', 'x': '127.103025712709', 'y': '37.5468565308905'}, 'address_name': '서울 광진구 광장동', 'address_typ

Geocoding unique addresses:   8%|█           | 95/1138 [00:00<00:06, 157.07it/s]

...

Geocoding unique addresses:  99%|█████████▉| 1125/1138 [00:07<00:00, 132.89it/s]

[Query] 경기도 안성시 미양면
[Status] 200
[JSON] {'documents': [{'address': {'address_name': '경기 안성시 미양면', 'b_code': '4155034000', 'h_code': '4155034000', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '경기', 'region_2depth_name': '안성시', 'region_3depth_h_name': '미양면', 'region_3depth_name': '미양면', 'sub_address_no': '', 'x': '127.217104719373', 'y': '36.9781308246344'}, 'address_name': '경기 안성시 미양면', 'address_type': 'REGION', 'road_address': None, 'x': '127.217104719373', 'y': '36.9781308246344'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}
[Query] 경기도 성남시 분당구 정자3동
[Status] 200
[JSON] {'documents': [{'address': {'address_name': '경기 성남시 분당구 정자3동', 'b_code': '', 'h_code': '4113557000', 'main_address_no': '', 'mountain_yn': 'N', 'region_1depth_name': '경기', 'region_2depth_name': '성남시 분당구', 'region_3depth_h_name': '정자3동', 'region_3depth_name': '', 'sub_address_no': '', 'x': '127.119553820185', 'y': '37.360763816811'}, 'address_name': '경기 성남시 분당구 정자3동', 'address_ty

Geocoding unique addresses: 100%|███████████| 1138/1138 [01:20<00:00, 14.15it/s]


[Exception] HTTPSConnectionPool(host='dapi.kakao.com', port=443): Max retries exceeded with url: /v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EB%85%B8%EC%9B%90%EA%B5%AC+%EC%A4%91%EA%B3%84%EB%B3%B8%EB%8F%99 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x172bb37a0>, 'Connection to dapi.kakao.com timed out. (connect timeout=None)'))


Applying cached coordinates: 100%|█| 14168294/14168294 [00:02<00:00, 6502895.55i


In [80]:
print(e_lat_list[:5])
print(e_lng_list[:5])

['37.5754222287938', '37.5754222287938', '37.5754222287938', '37.5754222287938', '37.5754222287938']
['126.96631967084', '126.96631967084', '126.96631967084', '126.96631967084', '126.96631967084']


In [90]:
#출발 주소 도착 주소 리스트 길이 확인: 둘 다 같은 길이로 확인. 위도 경도 추출 성공
print(len(s_lat_list))
print(len(e_lat_list))

14168294
14168294


In [94]:
df["출발 위도"] = s_lat_list
df["출발 경도"] = s_lng_list
df["도착 위도"] = e_lat_list
df["도착 경도"] = e_lng_list

In [106]:
df.head()

,출발 행정동 코드,도착 행정동 코드,출발 행정동 주소,도착 행정동 주소,출발 위도,출발 경도,도착 위도,도착 경도
0,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동,37.5754222287938,126.96631967084,37.5754222287938,126.96631967084
1,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동,37.5754222287938,126.96631967084,37.5754222287938,126.96631967084
2,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동,37.5754222287938,126.96631967084,37.5754222287938,126.96631967084
3,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동,37.5754222287938,126.96631967084,37.5754222287938,126.96631967084
4,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동,37.5754222287938,126.96631967084,37.5754222287938,126.96631967084


In [108]:
df['출발 위도'] = df['출발 위도'].astype(float)
df['출발 경도'] = df['출발 경도'].astype(float)
df['도착 위도'] = df['도착 위도'].astype(float)
df['도착 경도'] = df['도착 경도'].astype(float)

In [110]:
df['출발 위도'].min()

36.9382452357526

In [112]:
# 저장
df.to_csv("metropoli_lat_lng.csv", index=False, encoding="utf-8-sig")
print("✔️ 저장 완료: 당신의 파일_좌표추가_전체.csv")

✔️ 저장 완료: 당신의 파일_좌표추가_전체.csv


## heatmap 코딩 위한 데이터 준비 

In [123]:
df.head()

,출발 행정동 코드,도착 행정동 코드,출발 행정동 주소,도착 행정동 주소,출발 위도,출발 경도,도착 위도,도착 경도
0,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동,37.575422,126.96632,37.575422,126.96632
1,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동,37.575422,126.96632,37.575422,126.96632
2,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동,37.575422,126.96632,37.575422,126.96632
3,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동,37.575422,126.96632,37.575422,126.96632
4,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동,37.575422,126.96632,37.575422,126.96632


In [125]:
s_add = "서울특별시 종로구 사직동"
e_add = "서울특별시 종로구 사직동"

count = df[(df['출발 행정동 주소'] == s_add) & (df['도착 행정동 주소'] == e_add)].shape[0]
print(count)

3597
